In [2]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [4]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [5]:
data = pd.read_csv(r"/content/drive/MyDrive/Colab Notebooks/ML homework/Copy of default of credit card clients.csv")

In [6]:
data

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,1,20000,2,2,1,24,2,2,-1,-1,-2,-2,3913,3102,689,0,0,0,0,689,0,0,0,0,1
1,2,120000,2,2,2,26,-1,2,0,0,0,2,2682,1725,2682,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,3,90000,2,2,2,34,0,0,0,0,0,0,29239,14027,13559,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,4,50000,2,2,1,37,0,0,0,0,0,0,46990,48233,49291,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,5,50000,1,2,1,57,-1,0,-1,0,0,0,8617,5670,35835,20940,19146,19131,2000,36681,10000,9000,689,679,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,29996,220000,1,3,1,39,0,0,0,0,0,0,188948,192815,208365,88004,31237,15980,8500,20000,5003,3047,5000,1000,0
29996,29997,150000,1,3,2,43,-1,-1,-1,-1,0,0,1683,1828,3502,8979,5190,0,1837,3526,8998,129,0,0,0
29997,29998,30000,1,2,2,37,4,3,2,-1,0,0,3565,3356,2758,20878,20582,19357,0,0,22000,4200,2000,3100,1
29998,29999,80000,1,3,1,41,1,-1,0,0,0,-1,-1645,78379,76304,52774,11855,48944,85900,3409,1178,1926,52964,1804,1


# Helper Functions:

In [7]:
def preprocessing_(data):
    # get rid of missing values in MARRIAGE column
    data = data[data["MARRIAGE"] != 0]   
    
    # get rid of missing values in EDUCATION column
    data = data[data["EDUCATION"] != 0]    
    
    # discritize the AGE attribute to 6 interval
    data["AGE"] = pd.cut(data["AGE"], 6, labels=["AGE20", "AGE30", "AGE40", "AGE50", "AGE60", "AGE70"] )    
    #ordinal encoding
    encoder = OrdinalEncoder()
    # transform data
    data["AGE"] = encoder.fit_transform(np.array(data["AGE"]).reshape(-1, 1))
    
    return data

def print_cm(cm, labels, hide_zeroes=False, hide_diagonal=False, hide_threshold=None):
    """pretty print for confusion matrixes"""
    columnwidth = max([len(x) for x in labels] + [5])  # 5 is value length
    empty_cell = " " * columnwidth
    # Print header
    print("    " + empty_cell, end=" ")
    for label in labels:
        print("%{0}s".format(columnwidth) % label, end=" ")
    print()
    # Print rows
    for i, label1 in enumerate(labels):
        print("    %{0}s".format(columnwidth) % label1, end=" ")
        for j in range(len(labels)):
            cell = "%{0}.1f".format(columnwidth) % cm[i, j]
            if hide_zeroes:
                cell = cell if float(cm[i, j]) != 0 else empty_cell
            if hide_diagonal:
                cell = cell if i != j else empty_cell
            if hide_threshold:
                cell = cell if cm[i, j] > hide_threshold else empty_cell
            print(cell, end=" ")
        print()

        
def train_evaluate_val(model, x_train, y_train, folds, metrics):
    for metric in metrics:
        scores = cross_val_score(model, x_train, y_train, cv=folds, scoring = metric)
        print(f'{metric}:')
        print(f'Cross-validation: {scores}')
        print(f"Average: {scores.mean()}")
        print("----------------------------------------------------------")
    
def test_evaluate(model, x_test, y_test):
    prediction = model.predict(x_test)
    # prediction_prob = model.predict_proba(x_test)
    confusion = confusion_matrix(y_test, prediction)
    print("Confusion matrix",print_cm(confusion, ['Not pay', 'pay']))
    print('Accuracy: {:.2f}'.format(accuracy_score(y_test, prediction)))
    print('Precision: {:.2f}'.format(precision_score(y_test, prediction)))
    print('Recall: {:.2f}'.format(recall_score(y_test, prediction)))
    print('F1: {:.2f}'.format(f1_score(y_test, prediction)))
    print('AUC: {:.2f}'.format(roc_auc_score(y_test, prediction)))
    print(classification_report(y_test, prediction, target_names=['not pay', 'pay']))
    # precision_recall_curve(y_test, prediction_prob)

def optimize_model(model, x_train, y_train, parameters, metrics):
    for metric in metrics:
        print(f'{metric}:')
        grid_clf = GridSearchCV(model, param_grid = parameters, scoring = metric)
        grid_clf.fit(x_train, y_train)
        best_parameter = grid_clf.best_params_
        best_score = grid_clf.best_score_
        print(f"The best parameters: {best_parameter}\nThe best score: {best_score}\n")

# Prepare the data:

In [8]:
data = preprocessing_(data)

In [9]:
data

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,1,20000,2,2,1,0.0,2,2,-1,-1,-2,-2,3913,3102,689,0,0,0,0,689,0,0,0,0,1
1,2,120000,2,2,2,0.0,-1,2,0,0,0,2,2682,1725,2682,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,3,90000,2,2,2,1.0,0,0,0,0,0,0,29239,14027,13559,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,4,50000,2,2,1,1.0,0,0,0,0,0,0,46990,48233,49291,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,5,50000,1,2,1,3.0,-1,0,-1,0,0,0,8617,5670,35835,20940,19146,19131,2000,36681,10000,9000,689,679,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,29996,220000,1,3,1,1.0,0,0,0,0,0,0,188948,192815,208365,88004,31237,15980,8500,20000,5003,3047,5000,1000,0
29996,29997,150000,1,3,2,2.0,-1,-1,-1,-1,0,0,1683,1828,3502,8979,5190,0,1837,3526,8998,129,0,0,0
29997,29998,30000,1,2,2,1.0,4,3,2,-1,0,0,3565,3356,2758,20878,20582,19357,0,0,22000,4200,2000,3100,1
29998,29999,80000,1,3,1,2.0,1,-1,0,0,0,-1,-1645,78379,76304,52774,11855,48944,85900,3409,1178,1926,52964,1804,1


# combinning features and generate a high level features:

Combination one: combin all the preivous payements history of every person which will make a new feature represent the dealy of every person for pay his bills 

Make the new feature:("payment_history")

In [10]:
data["payment_delay"] = data['PAY_0'] + data['PAY_2'] + data['PAY_3'] + data['PAY_4'] + data['PAY_5'] + data['PAY_6'] 

In [11]:
data["payment_delay"]

0       -2
1        3
2        0
3        0
4       -2
        ..
29995    0
29996   -4
29997    8
29998   -1
29999    0
Name: payment_delay, Length: 29932, dtype: int64

Make the new feature:("payment_delay_amount")

In [12]:
data['payment_delay_amount'] = (data['BILL_AMT1'] - data['PAY_AMT1']) + (data['BILL_AMT2'] - data['PAY_AMT2']) + (data['BILL_AMT3'] - data['PAY_AMT3']) + (data['BILL_AMT4'] - data['PAY_AMT4']) + (data['BILL_AMT5'] - data['PAY_AMT5']) + (data['BILL_AMT6'] - data['PAY_AMT6'])

In [13]:
data['payment_delay_amount']

0          7015
1         12077
2         90635
3        222946
4         50290
          ...  
29995    682799
29996      6692
29997     39196
29998    119430
29999    222566
Name: payment_delay_amount, Length: 29932, dtype: int64

Make the new feature:("total_payment_amount")

In [14]:
data["total_payment_amount"] = data['PAY_AMT1'] + data['PAY_AMT2'] + data['PAY_AMT3'] + data['PAY_AMT4'] + data['PAY_AMT5'] + data['PAY_AMT6'] 

In [15]:
data["total_payment_amount"]

0           689
1          5000
2         11018
3          8388
4         59049
          ...  
29995     42550
29996     14490
29997     31300
29998    147181
29999      8308
Name: total_payment_amount, Length: 29932, dtype: int64

# Split the data:

In [16]:
y = data["default payment next month"]
data = data.drop("default payment next month", axis=1)

# x_train, x_test, y_train, y_test = train_test_split(data, y, test_size=0.2, random_state=42)
x_train, x_test, y_train, y_test = train_test_split(data[['LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'payment_delay', 'payment_delay_amount', 'total_payment_amount']], y, test_size=0.2, random_state=42)#'LIMIT_BAL' 'delay_amount'

Normalization:

In [17]:
scaler = preprocessing.StandardScaler().fit(np.array(x_train["payment_delay_amount"]).reshape(-1, 1))
x_train["payment_delay_amount"] = scaler.transform(np.array(x_train["payment_delay_amount"]).reshape(-1, 1))
x_test["payment_delay_amount"] = scaler.transform(np.array(x_test["payment_delay_amount"]).reshape(-1, 1))

scaler1 = preprocessing.StandardScaler().fit(np.array(x_train["total_payment_amount"]).reshape(-1, 1))
x_train["total_payment_amount"] = scaler1.transform(np.array(x_train["total_payment_amount"]).reshape(-1, 1))
x_test["total_payment_amount"] = scaler1.transform(np.array(x_test["total_payment_amount"]).reshape(-1, 1))

scaler2 = preprocessing.StandardScaler().fit(np.array(x_train["LIMIT_BAL"]).reshape(-1, 1))
x_train["LIMIT_BAL"] = scaler2.transform(np.array(x_train["LIMIT_BAL"]).reshape(-1, 1))
x_test["LIMIT_BAL"] = scaler2.transform(np.array(x_test["LIMIT_BAL"]).reshape(-1, 1))

x_train

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,payment_delay,payment_delay_amount,total_payment_amount
8759,-1.135178,2,1,2,0.0,-1,-0.556956,-0.409725
29948,0.945432,1,1,1,1.0,-4,-0.565613,0.169065
29155,-1.135178,2,2,1,3.0,1,-0.460194,-0.466397
11522,1.022492,2,1,1,1.0,-6,-0.655241,-0.461244
8242,0.328955,2,2,1,2.0,-9,-0.654879,-0.548229
...,...,...,...,...,...,...,...,...
29870,-1.212238,1,2,2,0.0,0,-0.507643,-0.467233
5402,-0.903999,1,1,2,1.0,0,-0.292742,-0.434501
863,-1.212238,2,2,1,1.0,0,-0.499167,-0.435128
15824,-0.672820,2,2,2,0.0,7,-0.601755,-0.469514


# Train & Evaluate: 

Helper Functions:

In [18]:
def print_cm(cm, labels, hide_zeroes=False, hide_diagonal=False, hide_threshold=None):
    """pretty print for confusion matrixes"""
    columnwidth = max([len(x) for x in labels] + [5])  # 5 is value length
    empty_cell = " " * columnwidth
    # Print header
    print("    " + empty_cell, end=" ")
    for label in labels:
        print("%{0}s".format(columnwidth) % label, end=" ")
    print()
    # Print rows
    for i, label1 in enumerate(labels):
        print("    %{0}s".format(columnwidth) % label1, end=" ")
        for j in range(len(labels)):
            cell = "%{0}.1f".format(columnwidth) % cm[i, j]
            if hide_zeroes:
                cell = cell if float(cm[i, j]) != 0 else empty_cell
            if hide_diagonal:
                cell = cell if i != j else empty_cell
            if hide_threshold:
                cell = cell if cm[i, j] > hide_threshold else empty_cell
            print(cell, end=" ")
        print()
        
def train_evaluate_val(model, x_train, y_train, folds, metrics):
    for metric in metrics:
        scores = cross_val_score(model, x_train, y_train, cv=folds, scoring = metric)
        print(f'{metric}:')
        print(f'Cross-validation: {scores}')
        print(f"Average: {scores.mean()}")
        print("----------------------------------------------------------")
    
def test_evaluate(model, x_test, y_test):
    prediction = model.predict(x_test)
    confusion = confusion_matrix(y_test, prediction)
    print("Confusion matrix",print_cm(confusion, ['Not pay', 'pay']))
    print('Accuracy: {:.2f}'.format(accuracy_score(y_test, prediction)))
    print('Precision: {:.2f}'.format(precision_score(y_test, prediction)))
    print('Recall: {:.2f}'.format(recall_score(y_test, prediction)))
    print('F1: {:.2f}'.format(f1_score(y_test, prediction)))
    print('AUC: {:.2f}'.format(roc_auc_score(y_test, prediction)))
    print(classification_report(y_test, prediction, target_names=['not pay', 'pay']))

def optimize_model(model, x_train, y_train, parameters, metrics):
    for metric in metrics:
        print(f'{metric}:')
        grid_clf = GridSearchCV(model, param_grid = parameters, scoring = metric)
        grid_clf.fit(x_train, y_train)
        best_parameter = grid_clf.best_params_
        best_score = grid_clf.best_score_
        print(f"The best parameters: {best_parameter}\nThe best score: {best_score}\n")

In [19]:
metrics = ["accuracy", 'precision', 'recall', 'f1', 'roc_auc']


Logistic Regression:

In [20]:
l_g_grid_values = {'C': [0.001, 0.01, 0.1, 1, 10, 100]}
l_g = LogisticRegression()

train and evaluate using cross validation:

In [21]:
train_evaluate_val(l_g, x_train, y_train, 5, metrics)

accuracy:
Cross-validation: [0.79369388 0.80037586 0.7993318  0.79661725 0.79766131]
Average: 0.7975360200459386
----------------------------------------------------------
precision:
Cross-validation: [0.640625   0.72340426 0.71929825 0.66539924 0.69166667]
Average: 0.6880786814287154
----------------------------------------------------------
recall:
Cross-validation: [0.15471698 0.1602262  0.15457116 0.16493874 0.15645617]
Average: 0.15818185051482228
----------------------------------------------------------
f1:
Cross-validation: [0.24924012 0.26234568 0.25446082 0.26435045 0.25518832]
Average: 0.2571170785574954
----------------------------------------------------------
roc_auc:
Cross-validation: [0.69449901 0.70601124 0.68665495 0.68453811 0.70046263]
Average: 0.694433187339832
----------------------------------------------------------


optimize the model using grid search: 

In [22]:
l_g = LogisticRegression()
optimize_model(l_g, x_train, y_train, l_g_grid_values, metrics)

accuracy:
The best parameters: {'C': 10}
The best score: 0.7975777824180413

precision:
The best parameters: {'C': 0.001}
The best score: 0.690950906535812

recall:
The best parameters: {'C': 10}
The best score: 0.15837035192858287

f1:
The best parameters: {'C': 10}
The best score: 0.25738502833713883

roc_auc:
The best parameters: {'C': 100}
The best score: 0.6944373337325617



Final evaluation on the test data:

In [23]:
l_g = LogisticRegression(C=10)
l_g.fit(x_train, y_train)
test_evaluate(l_g, x_test, y_test)

            Not pay     pay 
    Not pay  4552.0   108.0 
        pay  1090.0   237.0 
Confusion matrix None
Accuracy: 0.80
Precision: 0.69
Recall: 0.18
F1: 0.28
AUC: 0.58
              precision    recall  f1-score   support

     not pay       0.81      0.98      0.88      4660
         pay       0.69      0.18      0.28      1327

    accuracy                           0.80      5987
   macro avg       0.75      0.58      0.58      5987
weighted avg       0.78      0.80      0.75      5987



Decision Tree:

In [24]:
d_t = DecisionTreeClassifier()

In [25]:
d_t_grid_values = {'max_depth': [100, 20, 30, 50, 80, 100],
                    'criterion': ["gini", "entropy"],
                   'splitter': ["best", "random"],}

train and evaluate using cross validation:

In [26]:
train_evaluate_val(d_t, x_train, y_train, 5, metrics)

accuracy:
Cross-validation: [0.71413656 0.72374191 0.72562122 0.71538943 0.7218626 ]
Average: 0.7201503445395698
----------------------------------------------------------
precision:
Cross-validation: [0.37190813 0.38992042 0.375      0.36915078 0.375     ]
Average: 0.3761958663015355
----------------------------------------------------------
recall:
Cross-validation: [0.4009434  0.40622055 0.37794533 0.4005655  0.39585297]
Average: 0.3963055501218146
----------------------------------------------------------
f1:
Cross-validation: [0.3759058  0.39293798 0.37807183 0.37934186 0.38273921]
Average: 0.3817993376921711
----------------------------------------------------------
roc_auc:
Cross-validation: [0.59974676 0.61599802 0.60118767 0.5996365  0.60550049]
Average: 0.6044138872323763
----------------------------------------------------------


optimize the model using grid search: 

In [27]:
d_t = DecisionTreeClassifier()
                
optimize_model(d_t, x_train, y_train, d_t_grid_values, metrics)

accuracy:
The best parameters: {'criterion': 'entropy', 'max_depth': 20, 'splitter': 'random'}
The best score: 0.7533931927333473

precision:
The best parameters: {'criterion': 'entropy', 'max_depth': 20, 'splitter': 'random'}
The best score: 0.4246480023863602

recall:
The best parameters: {'criterion': 'gini', 'max_depth': 30, 'splitter': 'random'}
The best score: 0.394983728415699

f1:
The best parameters: {'criterion': 'gini', 'max_depth': 20, 'splitter': 'best'}
The best score: 0.391463857871809

roc_auc:
The best parameters: {'criterion': 'entropy', 'max_depth': 20, 'splitter': 'best'}
The best score: 0.6278238752109169



Final evaluation on the test data:

In [28]:
d_t = DecisionTreeClassifier(max_depth = 100, criterion = 'gini', splitter = 'random')
d_t.fit(x_train, y_train)
test_evaluate(d_t, x_test, y_test)

            Not pay     pay 
    Not pay  3793.0   867.0 
        pay   816.0   511.0 
Confusion matrix None
Accuracy: 0.72
Precision: 0.37
Recall: 0.39
F1: 0.38
AUC: 0.60
              precision    recall  f1-score   support

     not pay       0.82      0.81      0.82      4660
         pay       0.37      0.39      0.38      1327

    accuracy                           0.72      5987
   macro avg       0.60      0.60      0.60      5987
weighted avg       0.72      0.72      0.72      5987



Random Forest:

In [29]:
r_f = RandomForestClassifier()

In [30]:
r_f_grid_values = {'max_depth': [10, 20, 30, 50, 100],
                    'n_estimators': [10, 20, 40, 80, 100],
                    'criterion': ["gini", "entropy"]}

train and evaluate using cross validation:

In [31]:
train_evaluate_val(r_f, x_train, y_train, 5, metrics)

accuracy:
Cross-validation: [0.78930883 0.80329923 0.79912299 0.79432032 0.79891418]
Average: 0.796993109208603
----------------------------------------------------------
precision:
Cross-validation: [0.53821138 0.59967051 0.59369202 0.56587838 0.59628378]
Average: 0.5787472154495672
----------------------------------------------------------
recall:
Cross-validation: [0.31603774 0.34024505 0.31008483 0.31762488 0.332705  ]
Average: 0.32333949815944374
----------------------------------------------------------
f1:
Cross-validation: [0.4019025  0.43625378 0.40272615 0.41581786 0.42874396]
Average: 0.4170888472077233
----------------------------------------------------------
roc_auc:
Cross-validation: [0.73230228 0.76271752 0.73332827 0.73788001 0.74510885]
Average: 0.7422673859421544
----------------------------------------------------------


optimize the model using grid search: 

In [32]:
r_f = RandomForestClassifier()
                
optimize_model(r_f, x_train, y_train, r_f_grid_values, metrics)

accuracy:
The best parameters: {'criterion': 'gini', 'max_depth': 10, 'n_estimators': 80}
The best score: 0.807600751722698

precision:
The best parameters: {'criterion': 'entropy', 'max_depth': 10, 'n_estimators': 100}
The best score: 0.6293491544481786

recall:
The best parameters: {'criterion': 'gini', 'max_depth': 30, 'n_estimators': 100}
The best score: 0.3314468372663738

f1:
The best parameters: {'criterion': 'entropy', 'max_depth': 20, 'n_estimators': 80}
The best score: 0.41968958999507616

roc_auc:
The best parameters: {'criterion': 'entropy', 'max_depth': 10, 'n_estimators': 100}
The best score: 0.7672839296570468



Final evaluation on the test data:

In [33]:
r_f = RandomForestClassifier(max_depth = 20, n_estimators = 40, criterion = 'entropy')
r_f.fit(x_train, y_train)
test_evaluate(r_f, x_test, y_test)

            Not pay     pay 
    Not pay  4360.0   300.0 
        pay   933.0   394.0 
Confusion matrix None
Accuracy: 0.79
Precision: 0.57
Recall: 0.30
F1: 0.39
AUC: 0.62
              precision    recall  f1-score   support

     not pay       0.82      0.94      0.88      4660
         pay       0.57      0.30      0.39      1327

    accuracy                           0.79      5987
   macro avg       0.70      0.62      0.63      5987
weighted avg       0.77      0.79      0.77      5987



# Deep Neural network:

Metrics:

In [34]:
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [35]:
model = Sequential()
model.add(Dense(12, input_dim=8, activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(30, activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', precision_m, recall_m, f1_m,])
# fit the keras model on the dataset
model.fit(np.array(x_train), np.array(y_train), epochs=50, batch_size=100, validation_split=0.2)
model.evaluate(x=x_test, y=y_test)


Epoch 1/50
192/192 [==============================] - 17s 7ms/step - loss: 0.4999 - accuracy: 0.7977 - precision_m: 0.5931 - recall_m: 0.2825 - f1_m: 0.3688 - val_loss: 0.4545 - val_accuracy: 0.8094 - val_precision_m: 0.6205 - val_recall_m: 0.3692 - val_f1_m: 0.4568
Epoch 2/50
192/192 [==============================] - 0s 2ms/step - loss: 0.4614 - accuracy: 0.8058 - precision_m: 0.6111 - recall_m: 0.3312 - f1_m: 0.4217 - val_loss: 0.4523 - val_accuracy: 0.8066 - val_precision_m: 0.6332 - val_recall_m: 0.2997 - val_f1_m: 0.4006
Epoch 3/50
192/192 [==============================] - 1s 3ms/step - loss: 0.4560 - accuracy: 0.8055 - precision_m: 0.6310 - recall_m: 0.3313 - f1_m: 0.4247 - val_loss: 0.4506 - val_accuracy: 0.8106 - val_precision_m: 0.6294 - val_recall_m: 0.3615 - val_f1_m: 0.4529
Epoch 4/50
192/192 [==============================] - 1s 3ms/step - loss: 0.4611 - accuracy: 0.8020 - precision_m: 0.6065 - recall_m: 0.3180 - f1_m: 0.4095 - val_loss: 0.4497 - val_accuracy: 0.8068 - v

[0.4522141218185425,
 0.803741455078125,
 0.5924476981163025,
 0.29877156019210815,
 0.37035953998565674]